<center>

<h1>Final Assessment - Advanced Natural Language Processing</h1>

<i>

Course: 22DM015 Advanced Methods in Natural Language Processing <br>

Author(s): Ferran Boada Bergadà, Julián Romero, Lucia Sauer, Moritz Peist<br>

Programme: DSDM

<hr>

....

</i>

</center>

<hr>

## Setup

In [13]:
# Imports
import pandas as pd
import os

In [14]:
# Global constants
DATA_PATH = '../data/'
SPLITS = {'train': 'patent/train-00000-of-00001.parquet', 'validation': 'patent/validation-00000-of-00001.parquet', 'test': 'patent/test-00000-of-00001.parquet'}

## Data Load

In [15]:
# Data loading and persistence

def load_split(split_name, split_path, data_path="../data", dataset="ccdv/patent-classification"):
    """
    Load a specific split of the dataset, checking for local cache first.
    If the split is not cached locally, it will be downloaded and saved.
    Args:
        split_name (str): Name of the split (e.g., 'train', 'validation', 'test').
        split_path (str): Path to the split file in the dataset.
        data_path (str): Local path where the dataset is cached.
        dataset (str): Name of the dataset on Hugging Face Hub.
    Returns:
        pd.DataFrame: DataFrame containing the split data.
    """
    local_path = os.path.join(data_path, split_path)
    if os.path.exists(local_path):
        return pd.read_parquet(local_path)
    
    # Download and cache
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    df = pd.read_parquet(f"hf://datasets/{dataset}/{split_path}")
    df.to_parquet(local_path, index=False)
    return df

In [ ]:
df_train = load_split('train', SPLITS['train'])
df_validation = load_split('validation', SPLITS['validation'])
df_test = load_split('test', SPLITS['test'])

## Part 1

## Part 2

## Part 3

## Part 4